In [1]:
# File path
input_file = 'combined_shuffled_UOM_Data_sinhala_tamil.tsv'

# Counters
missing_source = 0
missing_target = 0
total_lines = 0

# Read file and skip header
with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()[1:]  # Skip header

    for line in lines:
        total_lines += 1
        parts = line.strip().split('\t')
        if len(parts) != 2:
            continue  # Skip malformed rows

        source, target = parts
        if not source.strip():
            missing_source += 1
        if not target.strip():
            missing_target += 1

# Summary
print(f"🔍 Total lines checked (excluding header): {total_lines}")
print(f"⚠️ Missing source (Tamil) sentences: {missing_source}")
print(f"⚠️ Missing target (Sinhala) sentences: {missing_target}")


🔍 Total lines checked (excluding header): 6705
⚠️ Missing source (Tamil) sentences: 0
⚠️ Missing target (Sinhala) sentences: 0


In [2]:
# File path
input_file = 'combined_shuffled_UOM_Data_sinhala_tamil.tsv'

# Set to store unique sentence pairs
unique_pairs = set()
duplicate_count = 0
total_lines = 0

# Read file and skip header
with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()[1:]  # Skip header

    for line in lines:
        total_lines += 1
        line = line.strip()
        if line in unique_pairs:
            duplicate_count += 1
        else:
            unique_pairs.add(line)

# Summary
print(f"📊 Total lines checked (excluding header): {total_lines}")
print(f"🔁 Duplicate sentence pairs: {duplicate_count}")
print(f"✅ Unique sentence pairs: {len(unique_pairs)}")


📊 Total lines checked (excluding header): 6705
🔁 Duplicate sentence pairs: 566
✅ Unique sentence pairs: 6139


In [3]:
# Input and output files
input_file = 'combined_shuffled_UOM_Data_sinhala_tamil.tsv'
duplicates_file = 'uom_duplicates.tsv'
without_duplicates_file = 'uom_parallel_corpus_without_duplicates.tsv'

# Dictionaries to track data
line_to_rows = {}     # line -> list of row numbers
duplicates = []
without_duplicates = []
seen_once = set()

# Read file
with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

header = lines[0].strip()
data_lines = lines[1:]

# Track line numbers (starting from 2 because line 1 is header)
for idx, line in enumerate(data_lines, start=2):
    line = line.strip()
    if line not in line_to_rows:
        line_to_rows[line] = [idx]
        without_duplicates.append(line)
    else:
        line_to_rows[line].append(idx)
        if line not in seen_once:
            duplicates.append(line)
            seen_once.add(line)

# ✅ Print duplicates with row numbers
print("🔁 Duplicate Sentence Pairs with Row Numbers:")
for dup in duplicates:
    rows = line_to_rows[dup]
    print(f"Rows {rows} → {dup}")

# 💾 Save duplicates for reference
with open(duplicates_file, 'w', encoding='utf-8') as df:
    df.write("row_numbers\tsource\ttarget\n")
    for dup in duplicates:
        source, target = dup.split('\t')
        row_numbers = ','.join(map(str, line_to_rows[dup]))
        df.write(f"{row_numbers}\t{source}\t{target}\n")

# 💾 Save without_duplicates data (only first occurrence)
with open(without_duplicates_file, 'w', encoding='utf-8') as cf:
    cf.write(header + '\n')
    for line in without_duplicates:
        cf.write(line + '\n')

# ✅ Summary
print(f"\n✅ Total duplicate sentence pairs: {len(duplicates)}")
print(f"📁 Duplicates saved to: {duplicates_file}")
print(f"📁 without_duplicates data saved to: {without_duplicates_file}")


🔁 Duplicate Sentence Pairs with Row Numbers:
Rows [7, 274, 340, 445, 481, 512, 556, 559, 637, 737, 814, 836, 915, 919, 937, 1032, 1073, 1150, 1242, 1305, 1410, 1466, 1565, 1643, 1714, 1732, 1772, 1792, 1799, 1859, 1876, 1949, 2016, 2071, 2184, 2293, 2382, 2422, 2461, 2634, 2658, 2696, 2781, 2879, 2906, 2926, 3015, 3210, 3247, 3317, 3353, 3596, 3621, 3735, 3761, 3951, 4033, 4061, 4092, 4162, 4266, 4315, 4349, 4511, 4536, 4551, 4585, 4587, 4593, 4600, 4642, 4840, 4879, 4889, 4904, 5067, 5262, 5272, 5357, 5535, 5564, 5674, 5687, 5706, 5738, 5942, 6226, 6241, 6341, 6375, 6416, 6454, 6575, 6589, 6638, 6657, 6661, 6696] → கெளரவ ரவி கருணாநாயக்க,	ගරු රවී කරුණානායක මහතා,
Rows [214, 330, 607, 616, 625, 815, 1141, 1327, 1369, 1432, 1566, 1865, 2018, 2200, 2517, 2649, 2738, 2838, 3199, 3574, 3690, 3693, 3780, 3782, 3859, 3873, 4014, 4086, 4104, 4482, 4651, 4890, 4891, 4906, 5290, 5327, 5466, 5606, 5658, 5824, 5924, 6335, 6501, 6678] → கெளரவ புத்திக பதிரண,	ගරු බුද්ධික පතිරණ මහතා,
Rows [232, 336, 35

In [4]:
import pandas as pd
import re
from collections import Counter
from itertools import chain

# Your input file path
input_file = 'uom_parallel_corpus_without_duplicates.tsv'

# Load parallel corpus
df = pd.read_csv(input_file, delimiter='\t', names=['source', 'target'], encoding='utf-8', on_bad_lines='skip')

# Regex pattern: allow Tamil \u0B80-\u0BFF, Sinhala \u0D80-\u0DFF, whitespace, and basic punctuation (.,!?) only
pattern = r'[^\u0B80-\u0BFF\u0D80-\u0DFF\s.,!?]'

# Function to detect if text contains unwanted characters
def has_unwanted_char(text):
    return bool(re.search(pattern, str(text)))

# Function to extract unwanted characters from text
def extract_unwanted_chars(text):
    return re.findall(pattern, str(text))

# Find rows with unwanted characters
source_unwanted_mask = df['source'].apply(has_unwanted_char)
target_unwanted_mask = df['target'].apply(has_unwanted_char)

print("Rows with unwanted chars in source:", source_unwanted_mask.sum())
print("Rows with unwanted chars in target:", target_unwanted_mask.sum())

# Extract unwanted characters from those rows
source_unwanted_chars = df.loc[source_unwanted_mask, 'source'].apply(extract_unwanted_chars)
target_unwanted_chars = df.loc[target_unwanted_mask, 'target'].apply(extract_unwanted_chars)

# Flatten lists and count frequency
source_counts = Counter(chain.from_iterable(source_unwanted_chars))
target_counts = Counter(chain.from_iterable(target_unwanted_chars))

# Convert to DataFrame and sort descending by count
source_df = pd.DataFrame(source_counts.items(), columns=['character', 'count']).sort_values(by='count', ascending=False)
target_df = pd.DataFrame(target_counts.items(), columns=['character', 'count']).sort_values(by='count', ascending=False)

# Save to TSV for review
source_df.to_csv('unwanted charactes/uom_unwanted_chars_source.tsv', sep='\t', index=False, encoding='utf-8')
target_df.to_csv('unwanted charactes/uom_unwanted_chars_target.tsv', sep='\t', index=False, encoding='utf-8')

print("Unwanted characters and counts saved to 'uom_unwanted_chars_source.tsv' and 'uom_unwanted_chars_target.tsv'")


Rows with unwanted chars in source: 2129
Rows with unwanted chars in target: 4343
Unwanted characters and counts saved to 'uom_unwanted_chars_source.tsv' and 'uom_unwanted_chars_target.tsv'


In [5]:
import pandas as pd
import re
from collections import Counter
from itertools import chain

input_file = 'uom_parallel_corpus_without_duplicates.tsv'

# Load file with skipping bad lines
df = pd.read_csv(input_file, delimiter='\t', names=['source', 'target'], encoding='utf-8', on_bad_lines='skip')

# Regex to match unwanted characters (not Tamil, Sinhala, space or basic punctuation)
pattern = r'[^\u0B80-\u0BFF\u0D80-\u0DFF\s.,!?]'

def has_unwanted_char(text):
    return bool(re.search(pattern, str(text)))

def extract_unwanted_chars(text):
    return re.findall(pattern, str(text))

# Filter rows with unwanted characters in source and target
source_unwanted_chars = df.loc[df['source'].apply(has_unwanted_char), 'source'].apply(extract_unwanted_chars)
target_unwanted_chars = df.loc[df['target'].apply(has_unwanted_char), 'target'].apply(extract_unwanted_chars)

# Combine all unwanted chars from both columns into one list
all_unwanted_chars = list(chain.from_iterable(source_unwanted_chars)) + list(chain.from_iterable(target_unwanted_chars))

# Count frequency of all unwanted characters combined
combined_counts = Counter(all_unwanted_chars)

# Convert to DataFrame
combined_df = pd.DataFrame(combined_counts.items(), columns=['character', 'count']).sort_values(by='count', ascending=False)

# Save to single TSV file
combined_df.to_csv('unwanted charactes/uom_unwanted_characters_combined.tsv', sep='\t', index=False, encoding='utf-8')

print("Combined unwanted characters saved to 'unwanted charactes/uom_unwanted_characters_combined.tsv'")


Combined unwanted characters saved to 'unwanted charactes/uom_unwanted_characters_combined.tsv'


In [6]:
import pandas as pd

input_file = 'uom_parallel_corpus_without_duplicates.tsv'
output_file = 'uom_parallel_corpus_second_stage.tsv'

# Load file and use the first row as header
df = pd.read_csv(input_file, delimiter='\t', encoding='utf-8', on_bad_lines='skip', header=0)

# Character to remove - the ZERO WIDTH JOINER (Unicode U+200D)
char_to_remove = '\u200d'

# Remove the character from both columns
df['source'] = df['source'].str.replace(char_to_remove, '', regex=False)
df['target'] = df['target'].str.replace(char_to_remove, '', regex=False)

# Save cleaned dataframe with header
df.to_csv(output_file, sep='\t', index=False, encoding='utf-8')

print(f"Removed '{repr(char_to_remove)}' from both columns and saved to '{output_file}'")


Removed ''\u200d'' from both columns and saved to 'uom_parallel_corpus_second_stage.tsv'


# Manuvaly Removed other Unwanted Characters